# 数据集准备
### 标准多人标注文件解压
解压操作前请将OSS中
* annotations文件夹中的压缩包放置于data文件夹下
* image文件夹中的压缩包放置于data/image文件夹下

#### 解压标注文件

In [4]:
import shutil
import zipfile
import os

fileList = os.listdir('../data')
fileList.remove('image')
# fileList.remove('identify')

for file in fileList:
	if file.split('.')[-1] != 'zip':
		continue
	task = file.split('.')[0]
	ZipFile = zipfile.ZipFile('../data/%s.zip' % task)
	ZipFile.extract(ZipFile.namelist()[0], '../data')
	ZipFile.close()
	shutil.move('../data/annotations/instances_default.json', '../data/%s.json' % task)
	shutil.rmtree('../data/annotations')
	os.remove('../data/%s.zip' % task)

#### 解压图片文件

In [3]:
import zipfile
import os

fileList = os.listdir('../data/image')

tasks = []
for file in fileList:
	if file.split('.')[-1] != 'zip':
		continue
	task = file.split('.')[0]
	tasks.append(task)
	ZipFile = zipfile.ZipFile('../data/image/%s.zip' % task)
	ZipFile.extractall('../data/image')
	ZipFile.close()
	os.remove('../data/image/%s.zip' % task)

### 处理C系列数据不规范命名

In [5]:
import os
import shutil

folders = ['task1', 'task2', 'task3', 'task4', 'task5']
for i in range(len(folders)):
	folder=folders[i]
	if folder == '':
		continue
	folder = folders[i]
	fileList = os.listdir('../data/image/%s' % folder)
	for file in fileList:
		shutil.move('../data/image/%s/%s' % (folder, file), '../data/image/C%i-%s' % (i+1, file))
	os.removedirs('../data/image/%s' % folder)

In [2]:
import ujson as json

files = ['', '', 'taskc3.json']
for j in range(len(files)):
	file = files[j]
	if file == '':
		continue
	with open('../data/%s' % file, 'r') as f:
		data = json.load(f)
	for i in range(len(data['images'])):
		data['images'][i]['file_name'] = ('C%i-' % (j+1)) + data['images'][i]['file_name'].split('/')[-1]
	with open('../data/%s' % file, 'w') as f:
		json.dump(data, f)

### 数据集合并

In [11]:
import json
import os

with open('../data/task0.json', 'r') as file:
	jsonFile = json.load(file)
fileList = os.listdir('../data')
fileList.remove('image')
fileList.remove('yolo')
# fileList.remove('identify')
if 'task.json' in fileList:
	fileList.remove('task.json')
fileList.remove('task0.json')
for file in fileList:
	names = file.split('.')
	if names[1] != 'json':
		continue
	task = names[0]
	with open("../data/%s.json" % task, 'r') as file:
		nowFile = json.load(file)
	nowImageId = len(jsonFile['images'])
	nowAnnId = len(jsonFile['annotations'])
	for i in range(len(nowFile['images'])):
		nowFile['images'][i]['id'] += nowImageId
	for i in range(len(nowFile['annotations'])):
		nowFile['annotations'][i]['id'] += nowAnnId
		nowFile['annotations'][i]['image_id'] += nowImageId
	jsonFile['images'] += nowFile['images']
	jsonFile['annotations'] += nowFile['annotations']
	print(nowAnnId)
	print(nowImageId)
with open("../data/task.json", 'w') as f:
    f.write(json.dumps(jsonFile))

3676
2907
4176
3407
6107
3907
6607
4407
6903
4907
7921
5407
8732
5907
9616
6407
10116
6907
11699
7209
12199
7709
12676
8209
13096
8709
13573
9209
13799
9709
17012
10209
17124
10709
17534
11209
18085
11709
18585
12209
19085
12709
19544
13209
19988
13709


### 平衡数据集中标注个数

In [6]:
from pycocotools.coco import COCO
import random
import json

# 设置目标标注个数
targetNumber = 800

coco = COCO('../data/task.json')
cocoIds = coco.getCatIds()
removeCat = {}
removeList = []
for it in cocoIds:
	length = len(coco.getImgIds(catIds=[it]))
	if length > targetNumber:
		removeCat[it] = length - targetNumber
print(removeCat)
removeKeys = list(removeCat.keys())
for it in removeKeys:
	images = coco.getImgIds(catIds=[it])
	random.shuffle(images)
	for image in images:
		annIds = coco.getAnnIds(imgIds=[image])
		willRemove = True
		for annId in annIds:
			if coco.loadAnns(annId)[0]['category_id'] not in removeCat.keys():
				willRemove = False
				break
		if willRemove:
			continue
		removeList.append(image)
		removeCat[it] -= 1
		removeCatKeysWillRemove = []
		for k, v in removeCat.items():
			if v <= 0:
				removeCatKeysWillRemove.append(k)
		for removeKey in  removeCatKeysWillRemove:
			removeCat.pop(removeKey)
		if it not in removeCat.keys() or removeCat[it] <= 0:
			break
print(removeCat)
print(len(removeList))

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
{1: 126, 5: 105, 9: 422, 10: 196, 12: 1812}
{1: 126, 5: 105, 9: 422, 10: 196, 12: 1812}
0


In [2]:
# 删除多余标注
with open('../data/task.json', 'r') as file:
	jsonFile = json.load(file)
i = 0
while i < len(jsonFile['images']):
    if jsonFile['images'][i]['id'] in removeList:
        jsonFile['images'].pop(i)
        i -= 1
    i += 1
i = 0
while i < len(jsonFile['annotations']):
    if jsonFile['annotations'][i]['image_id'] in removeList:
        jsonFile['annotations'].pop(i)
        i -= 1
    i += 1
with open("../data/task.json", 'w') as f:
    f.write(json.dumps(jsonFile))

NameError: name 'json' is not defined

In [3]:
# 删除多余图片
coco = COCO('../data/task.json')
removeList = []
imgIds = coco.getImgIds()
for imgId in imgIds:
	if len(coco.getAnnIds(imgIds=[imgId])) == 0:
		removeList.append(imgId)
with open('../data/task.json', 'r') as file:
	jsonFile = json.load(file)
i = 0
while i < len(jsonFile['images']):
    if jsonFile['images'][i]['id'] in removeList:
        jsonFile['images'].pop(i)
        i -= 1
    i += 1
i = 0
while i < len(jsonFile['annotations']):
    if jsonFile['annotations'][i]['image_id'] in removeList:
        jsonFile['annotations'].pop(i)
        i -= 1
    i += 1
with open("../data/task.json", 'w') as f:
    f.write(json.dumps(jsonFile))

NameError: name 'COCO' is not defined

### 移除无标注文件

In [4]:
from pycocotools.coco import COCO


coco = COCO('../data/task.json')
isAnn = []
anns = coco.loadAnns(coco.getAnnIds())
imgs = coco.loadImgs(coco.getImgIds())
for it in anns:
	idx = it['image_id']
	isAnn.append(idx)
removeList = []
for it in imgs:
	if it['id'] not in isAnn:
		removeList.append(it['id'])
len(removeList)

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


0

In [5]:
import json

with open('../data/task.json', 'r') as file:
	jsonFile = json.load(file)
i = 0
print(len(jsonFile['images']))
while i < len(jsonFile['images']):
    if jsonFile['images'][i]['id'] in removeList:
        jsonFile['images'].pop(i)
        i -= 1
    i += 1
print(len(jsonFile['images']))
i = 0
while i < len(jsonFile['annotations']):
    if jsonFile['annotations'][i]['image_id'] in removeList:
        jsonFile['annotations'].pop(i)
        i -= 1
    i += 1
with open("../data/task.json", 'w') as f:
    f.write(json.dumps(jsonFile))

12477
12477


### 转为yolo格式数据集

In [12]:
from coco_to_yolo import ConvertCOCOToYOLO

ConvertCOCOToYOLO(img_path='../data/image', json_path='../data/task.json').convert()


loading annotations into memory...
Done (t=0.29s)
creating index...
index created!


## 切换测量/识别赛项工作空间
### 切换到测量赛项

In [10]:
import shutil
import os

files = os.listdir('../data')
if 'identify' in files:
	files.remove('identify')
else:
	os.mkdir('../data/identify')
if 'measure' in files:
	files.remove('measure')
else:
	os.mkdir('../data/measure')
for file in files:
	shutil.move(os.path.join('../data', file), os.path.join('../data/identify', file))
files = os.listdir('../data/measure')
for file in files:
	shutil.move(os.path.join('../data/measure', file), os.path.join('../data', file))

### 切换到识别赛项

In [7]:
import shutil
import os

files = os.listdir('../data')
if 'identify' in files:
	files.remove('identify')
else:
	os.mkdir('../data/identify')
if 'measure' in files:
	files.remove('measure')
else:
	os.mkdir('../data/measure')
files.remove('identify')
for file in files:
	shutil.move(os.path.join('../data', file), os.path.join('../data/measure', file))
files = os.listdir('../data/identify')
for file in files:
	shutil.move(os.path.join('../data/identify', file), os.path.join('../data', file))